In [3]:
import pymongo
import json

In [6]:
config_file = open("../config.cfg")
config = json.load(config_file)
client = pymongo.MongoClient(config["mongodb_uri"])
db = client[config["database"]]
current_members = db["current_ls_members"]
archive_members = db["archive_ls_members"]

In [36]:
count = 0
for row in current_members.find({}):
    if archive_members.count_documents({"_id":row["_id"]}) > 0:
        continue
        count = count+1
        old_row = archive_members.find_one({"_id":row["_id"]})
        current_sessions = []
        for session in old_row["session"]:
            current_sessions.append({"session" : session,
                                    "party" : "NA",
                                    "constituency" : "NA"
                                   })
        current_sessions.append({"session" : "17",
                                "party" : row["party"],
                                "constituency" : row["constituency"],
                                "state" : row["state"]
                               })
        row["session"] = current_sessions
        current_members.update_one({"_id":row["_id"]},{"$set":{"session":current_sessions}})
    else:
        current_sessions = []
        current_sessions.append({"session" : "17",
                                "party" : row["party"],
                                "constituency" : row["constituency"],
                                "state" : row["state"]
                               })
        current_members.update_one({"_id":row["_id"]},{"$set":{"session":current_sessions}})